<a href="https://colab.research.google.com/github/HYUNMIN-HWANG/Colab/blob/main/DACON_vision2/vision2_0301_1_vision1train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


# vision1에 나온 알파벳들을 훈련시켜보자

In [3]:
# warnings.filterwarnings("ignore")
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
import cv2
import gc
from keras import backend as bek
from sklearn.model_selection import train_test_split, KFold
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV



In [4]:
train = pd.read_csv('/content/MyDrive/MyDrive/AIA/DACON_vision1/train.csv')
print(train.shape)  # (2048, 787)

submission = pd.read_csv('/content/MyDrive/MyDrive/AIA/DACON_vision1/submission.csv')
print(submission.shape) # (20480, 2)

test = pd.read_csv('/content/MyDrive/MyDrive/AIA/DACON_vision1/test.csv')
print(test.shape)   # (20480, 786)

(2048, 787)
(20480, 2)
(20480, 786)


In [5]:
x = np.load('/content/MyDrive/MyDrive/Colab/DACON_vision2/npy/vision1_x28.npy')
print(x.shape)
y = np.load('/content/MyDrive/MyDrive/Colab/DACON_vision2/npy/vision1_y28.npy')
print(y.shape)

(22528, 56, 56, 3)
(22528,)


In [6]:
# modeling
def modeling() :
    model = Sequential()
    model.add(Conv2D(128, 3, activation='selu', padding='same', input_shape=(xx_train.shape[1], xx_train.shape[2], xx_train.shape[3])))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(2,2))

    model.add(Conv2D(32, 3, activation='selu', padding='same'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(32, 2, activation='elu', padding='same'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(2,2))

    model.add(Conv2D(64, 3, activation='elu', padding='same'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(64, 2, activation='elu', padding='same'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(2,2))

    model.add(Flatten())

    model.add(Dense(256, activation='selu'))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='selu'))
    model.add(BatchNormalization())
    model.add(Dense(26, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.003), metrics=['acc'])
    return model

In [ ]:
kf_times = 8
kf = KFold(n_splits=kf_times, shuffle=True, random_state=42)

idg = ImageDataGenerator(height_shift_range=(-1,1), 
                        width_shift_range=(-1,1),
                        rotation_range=120,
                        fill_mode='nearest'
                        )
idg2 = ImageDataGenerator()

kf_n = 1
loss_hist = []
acc_hist = []
for train_index, test_index in kf.split(x, y) :
    print("\n** Kfold %d 번째 실행 중 **" % kf_n)
    xx_train, xx_test = x[train_index], x[test_index]
    yy_train, yy_test = y[train_index], y[test_index]

    xx_train, xx_valid, yy_train, yy_valid = train_test_split(xx_train, yy_train, train_size=0.9, shuffle=True, random_state=42)
    print(xx_train.shape, xx_test.shape, xx_valid.shape)    # (16219, 56, 56, 3) (4506, 56, 56, 3) (1803, 56, 56, 3)
    print(yy_train.shape, yy_test.shape, yy_valid.shape)    # (16219,) (4506,) (1803,)

    train_generator = idg.flow(xx_train, yy_train, batch_size=16)
    test_generator = idg2.flow(xx_test, yy_test, batch_size=16)
    valid_generator = idg2.flow(xx_valid, yy_valid)

    model = modeling()

    path = '/content/MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0301_1_vison1_{val_loss:.4f}.hdf5'
    cp = ModelCheckpoint(path, monitor='val_loss', save_best_only=True, mode='min')
    rl = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, mode='min')
    es = EarlyStopping(monitor='val_loss', patience=20, mode='min')

    model.fit_generator(train_generator, epochs=500, validation_data=valid_generator, callbacks=[cp, rl, es])

    loss, acc = model.evaluate(test_generator)
    print("loss : ", loss)
    print("acc : ", acc)

    loss_hist.append(loss)
    acc_hist.append(acc)

    kf_n +=1

print("mean loss : ", sum(loss_hist)/kf_times)
print("mean acc : ", sum(acc_hist)/kf_times)


** Kfold 1 번째 실행 중 **
(17740, 56, 56, 3) (2816, 56, 56, 3) (1972, 56, 56, 3)
(17740,) (2816,) (1972,)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/500
1109/1109 [==============================] - 24s 21ms/step - loss: 2.3984 - acc: 0.3093 - val_loss: 1.6198 - val_acc: 0.5193
Epoch 2/500
1109/1109 [==============================] - 22s 20ms/step - loss: 1.4072 - acc: 0.5613 - val_loss: 1.1043 - val_acc: 0.6537
Epoch 3/500
1109/1109 [==============================] - 22s 20ms/step - loss: 1.1948 - acc: 0.6223 - val_loss: 0.8450 - val_acc: 0.7368
Epoch 4/500
1109/1109 [==============================] - 22s 20ms/step - loss: 1.0819 - acc: 0.6558 - val_loss: 0.8065 - val_acc: 0.7333
Epoch 5/500
1109/1109 [==============================] - 23s 20ms/step - loss: 1.0040 - acc: 0.6855 - val_loss: 0.8117 - val_acc: 0.7378
Epoch 6/500
1109/1109 [==============================] - 22s 20ms/step - loss: 0.9608 - acc: 0.6972 - val_loss: 0.7402 - val_acc: 0.7703
Epoch 7/500
1109/1109 [==============================] - 22s 20ms/step - loss: 0.9338 - acc: 0.7042 - val_loss: 0.6589 - val_acc: 0.7911
Epoch 8/500
1109/1109 [==================

In [ ]:
# 잘 예측하는지 확인 
import matplotlib.pyplot as plt
import matplotlib.image as img
from glob import glob 

# submission
sub = pd.read_csv('/content/MyDrive/MyDrive/Colab/DACON_vision2/sample_submission.csv')
# print(sub.head())

# model = load_model('/content/MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0226_1_vison1_0.3941.hdf5') # <<-- 여기에 데이콘2 테스트 데이터를 넣어서 숫자를 예측한다.

n = 50000
row = 0
for i in range(5000) :
    print("start >>> ", n)
    test_img=glob(f'/content/MyDrive/MyDrive/Colab/DACON_vision2/contour/{n}_*.png')
    print(len(test_img))

    predict_list = []

    for img in test_img :
        pred_img = cv2.imread(img) # 

        # print(pred_img.shape)   # (64, 64, 3)
        plt.imshow(pred_img)
        cv2.waitKey(0)

        pred_img = pred_img.astype('float32')
        pred_img = cv2.resize(pred_img, (56,56))
        # print(pred_img.shape)   # (56, 56)

        pred_img = pred_img.reshape(1, pred_img.shape[0], pred_img.shape[1], pred_img.shape[2])/255.
        # print(pred_img.shape)   # (1, 56, 56, 3)
        pred_generator = idg2.flow(pred_img, shuffle=False)

        result = model.predict_generator(pred_generator, verbose=True)
        print(result.max())
        if result.max() < 0.6 : # 최대 예측값이 60%를 넘지 않는다면 이상한 문자이므로 패쓰한다.
            pass
        else :
            result_max = result.argmax(1)[0]
            print(result_max) 
            predict_list.append(result_max)

    # 중복값 제거하기 위해서 집합으로 만들었다가 리스트로 변환
    predict_set = set(predict_list)
    predict_list = list(predict_set)
    print(predict_list)

    # submission 형태로 만들기
    result = np.zeros((1,26), dtype=np.int8)
    # print(result)

    for idx in predict_list :
        result[0][idx] = 1
    print(result)

    sub.iloc[row][1:] = result[0]
    sub.to_csv('/content/MyDrive/MyDrive/Colab/DACON_vision2/sub_0301_1.csv', index=False)
    print(sub.head())

    n += 1
    row += 1

print("==Done==")